## **Prerequisites**

#### 1. Setup Azure Machine Learning Workspace via the Portal
- Note workspace name, resource group, and subscription id
#### 2. Create and populate .env file in the home directory
- Use [.sample.env](../.sample.env) as a guide
#### 3. Create and activate conda virtual env
- Run the following bash commands via the terminal
```bash
    conda env create --name many_models --file=../environment/conda.yaml
    conda activate many_models
    az login
```
- Select the many_models python interpreter and kernel to run the remainder of this notebook
#### 4. Load environment variables by running the cell below

In [5]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

#Confirm variabels were loaded
print(os.getenv("WORKSPACE_NAME"))

mm-aml-wksp


#### 5. Load dataset to AML workspace

In [6]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

import os

# authenticate
credential = DefaultAzureCredential(tenantid=os.environ.get('TENANT_ID'))

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id = os.environ.get('SUBSCRIPTION_ID'),
    resource_group_name = os.environ.get('RESOURCE_GROUP_NAME'),
    workspace_name = os.environ.get('WORKSPACE_NAME'),
)

In [ ]:
from azureml.opendatasets import OjSalesSimulated
import os

DATASET_MAX_FILES = 15 # edit this field to download up to 11,973 files (Each file is a SKU/Store combination; You will fit 1 model per file)

local_path = "../data/oj_sim_sales"
if not os.path.exists(local_path):
    os.mkdir(local_path)

oj_sales_files = OjSalesSimulated.get_file_dataset().take(DATASET_MAX_FILES)
oj_sales_files.download(local_path, overwrite=True)

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import time 

# set the version number of the data asset to the current UTC time
v1 = time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())
data_name = "oj-simulated-sales-data"
my_data = Data(
    name=data_name,
    version=v1,
    description="Chicago area orange juice sales data",
    path=local_path,
    type=AssetTypes.URI_FOLDER,
)

# create data asset
ml_client.data.create_or_update(my_data)

print(f"Data asset created. Name: {my_data.name}, version: {my_data.version}")

In [ ]:
import pandas as pd

data_asset = ml_client.data.get(data_name, label="latest")
print(data_asset.path)
df = pd.read_csv(data_asset.path)
display(df.head(10))